<a href="https://colab.research.google.com/github/vidulakamat/FB_Posts_SC_Analysis/blob/main/Count_Antibodies_Post_Plus_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import re
import spacy
import time

In [ ]:
import collections

In [ ]:
# Import stopwords
import nltk
from nltk.corpus import stopwords

# Import textblob
from textblob import Word, TextBlob

In [ ]:
pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.4.1.2)
  Using cached https://files.pythonhosted.org/packages/98/87/ef312eef26f5cecd8b17ae9654cdd8d1fae1eb6dbd87257d6d73c128a4d0/transformers-4.3.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fd/5b/44baae602e0a30bcc53fbdbc60bd940c15e143d252d658dfdefce736ece5/tokenizers-0.10.1-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: aspect-based-sentiment-analysis 2.0.2 has requirement transformers==2.5, but you'll have transformers 4.3.2 which is incompatible.
  Found existing installation: tokenizers 0.5.0
    Uninstalling tokenizers-0.5.0:
      Successfully uninstalled tokenizers-0.5.0
  Found existing installation: transformers 2.5.0
    Uninstalling transformers-2.5.0:
      Successfully uninstalled transformers-2.5.0


In [ ]:
pip install kneed

In [ ]:
pip install aspect-based-sentiment-analysis

  Using cached https://files.pythonhosted.org/packages/04/58/3d789b98923da6485f376be1e04d59ad7003a63bdb2b04b5eea7e02857e5/transformers-2.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/1d/ea7e2c628942e686595736f73678348272120d026b7acd54fe43e5211bb1/tokenizers-0.5.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: sentence-transformers 0.4.1.2 has requirement transformers<5.0.0,>=3.1.0, but you'll have transformers 2.5.0 which is incompatible.
  Found existing installation: tokenizers 0.10.1
    Uninstalling tokenizers-0.10.1:
      Successfully uninstalled tokenizers-0.10.1
  Found existing installation: transformers 4.3.2
    Uninstalling transformers-4.3.2:
      Successfully uninstalled transformers-4.3.2


In [ ]:
from kneed import KneeLocator

In [ ]:
import aspect_based_sentiment_analysis as absa

In [ ]:
df = pd.read_excel('SC_Posts_1.xls', header=None)
df.columns = ['post']

df.shape

(17946, 1)

In [ ]:
df = df[(df['post'].str.find('FALSE')==-1)]
df = df[(df['post'].str.find('Comments')==-1)]
df.shape

(12274, 1)

In [ ]:
# function to preprocess speech
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    
    #text.replace('[^a-zA-Z]', '')
    #text.replace('[^\w\s]', '')
    #text = re.sub('[^a-zA-Z]', ' ', text)
    #text = re.sub(r'\s+', ' ', text)
    
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
df['post_clean'] = df['post'].apply(clean)
df.shape

(12274, 2)

In [ ]:
def preprocess_sent(tweet):
    processed_tweet = tweet
    processed_tweet.replace('[^a-zA-Z]', '')
    processed_tweet.replace('[^\w\s]', '')
    processed_tweet = re.sub('[^a-zA-Z]', ' ', processed_tweet)
    processed_tweet = re.sub(r'\s+', ' ', processed_tweet)
    return(processed_tweet)

df['Processed_Sent'] = df['post_clean'].apply(lambda x: preprocess_sent(x))
df.head()

,post,post_clean,Processed_Sent
0,Shannon Murray Gormley,Shannon Murray Gormley,Shannon Murray Gormley
2,Just wanted to let anyone know if you are able...,Just wanted to let anyone know if you are able...,Just wanted to let anyone know if you are able...
4,Alicia Gottschalk,Alicia Gottschalk,Alicia Gottschalk
5,My hubby had it last week and it has done wond...,My hubby had it last week and it has done wond...,My hubby had it last week and it has done wond...
7,Laura Sutera,Laura Sutera,Laura Sutera


In [ ]:
df = df[df['Processed_Sent'].str.strip().str.len()>0]
df.shape

(11557, 3)

In [ ]:
# load english language model
nlp = spacy.load('en_core_web_sm')

In [ ]:
# function to retrieve FIRST person name from post

def get_person_name(text):
    
    doc = nlp(text)
    
    sent = []
    
    persons_count = 0
    person_names = []
    prev_person_name_index = -1
    token_count = 0
    
    for token in doc:
        if (token.ent_type_=='PERSON'):
          persons_count+= 1
          if(prev_person_name_index == -1):
            person_names.append(token.text)
            prev_person_name_index = token_count
          else:
            if((token_count-prev_person_name_index)==1):
              person_names.append(token.text)
              prev_person_name_index = token_count
        token_count += 1

    person_name = ' '.join(person_names)

    return(person_name)

In [ ]:
# function for checking if post contains only person names
def check_only_person_names(text):
    
    doc = nlp(text)
    
    sent = []
    
    persons_count = 0
    for token in doc:
        if (token.ent_type_=='PERSON'):
          persons_count+= 1

    return((persons_count == len(doc)))

In [ ]:
df['Person_Name'] = df['Processed_Sent'].apply(lambda x: get_person_name(x))
df['Is_Only_Person_Name'] = df['Processed_Sent'].apply(lambda x: check_only_person_names(x))

In [ ]:
df.head()

,post,post_clean,Processed_Sent,Person_Name,Is_Only_Person_Name
0,Shannon Murray Gormley,Shannon Murray Gormley,Shannon Murray Gormley,Shannon Murray Gormley,True
2,Just wanted to let anyone know if you are able...,Just wanted to let anyone know if you are able...,Just wanted to let anyone know if you are able...,,False
4,Alicia Gottschalk,Alicia Gottschalk,Alicia Gottschalk,Alicia Gottschalk,True
5,My hubby had it last week and it has done wond...,My hubby had it last week and it has done wond...,My hubby had it last week and it has done wond...,,False
7,Laura Sutera,Laura Sutera,Laura Sutera,Laura Sutera,True


In [ ]:
df.sample(20)

,post,post_clean,Processed_Sent,Person_Name,Is_Only_Person_Name
10249,Sally Billett same. Yesterday morning I was so...,Sally Billett same. Yesterday morning I was so...,Sally Billett same Yesterday morning I was so ...,Sally Billett,False
17213,"Donna Cordner , that's fine. But I'm also happ...","Donna Cordner , that fine. But I'm also happy ...",Donna Cordner that fine But I m also happy to ...,Donna Cordner,False
6203,Best wishes to your family and hoping your mom...,Best wishes to your family and hoping your mom...,Best wishes to your family and hoping your mom...,Don,False
9398,Peter Moore,Peter Moore,Peter Moore,Peter Moore,True
17555,Questions about this.,Questions about this.,Questions about this,,False
9886,Michelle Bell well that works!!! I wish I had ...,Michelle Bell well that works!!! I wish I had ...,Michelle Bell well that works I wish I had one...,Michelle Bell,False
5302,My heart is with your family!,My heart is with your family!,My heart is with your family,,False
4628,Mary Paul Taylor thank you! I can definitely a...,Mary Paul Taylor thank you! I can definitely a...,Mary Paul Taylor thank you I can definitely as...,Mary Paul Taylor,False
6794,Didi Victorious Gray,Didi Victorious Gray,Didi Victorious Gray,,False
4497,Deborah Champagne,Deborah Champagne,Deborah Champagne,Deborah Champagne,True


In [ ]:
# Total number of people present in post (those who posted/commented + those referred in post by others)

total_number_of_people = len(set(list(df['Person_Name'])))
total_number_of_people

2845

In [ ]:
# Removing those entries where only person names are present in post

df = df[df['Is_Only_Person_Name']==False]

In [ ]:
# Removing duplicate posts

df = df.drop_duplicates(subset = ["Processed_Sent"])

In [ ]:
# Final posts to be considered for similarities and further processing

df.shape

(5290, 5)

In [ ]:
df.sample(10)

,post,post_clean,Processed_Sent,Person_Name,Is_Only_Person_Name
13873,Marcy Buchanan they use it in Peru to prevent ...,Marcy Buchanan they use it in Peru to prevent ...,Marcy Buchanan they use it in Peru to prevent ...,Marcy Buchanan,False
9008,#PrayingForYou ??,#PrayingForYou ??,PrayingForYou,,False
20,Adriane Hogan McCullagh first 10 days of symptoms,Adriane Hogan McCullagh first 10 days of symptoms,Adriane Hogan McCullagh first days of symptoms,,False
2800,I'm glad you're here to tell your story! Take ...,I'm glad you're here to tell your story! Take ...,I m glad you re here to tell your story Take i...,,False
4511,It was also politically tabu at the time to sa...,It was also politically tabu at the time to sa...,It was also politically tabu at the time to sa...,,False
4246,Somebody that came from Europe cough pretty cl...,Somebody that came from Europe cough pretty cl...,Somebody that came from Europe cough pretty cl...,,False
15937,Jennifer Merryman Cartwright,Jennifer Merryman Cartwright,Jennifer Merryman Cartwright,Jennifer Merryman,False
14982,Jayne Ann Mitchell how was your recovery after...,Jayne Ann Mitchell how was your recovery after...,Jayne Ann Mitchell how was your recovery after...,Jayne Ann Mitchell,False
11884,"it's actually for my sister, but we wanted to ...","it actually for my sister, but we wanted to be...",it actually for my sister but we wanted to be ...,,False
9457,Phylica Jones Frankenberger if you dont take i...,Phylica Jones Frankenberger if you dont take i...,Phylica Jones Frankenberger if you dont take i...,Phylica Jones,False


In [ ]:
# Column to be utilized is Processed_Sent

In [ ]:
unique_posts = list(df['Processed_Sent'])
len(unique_posts)

5290

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2')

OSError: ignored

In [ ]:
queries = ["received Infusion", "received antibodies", "received regeneron", "received eli lilly", "received bamlanivimab"]
passages = list(df['Processed_Sent'])

In [ ]:
matching_queries_dict = {}
matching_posts_count = 0

#Search in a loop for the individual queries
for query in queries:
    matching_tuples = []
    start_time = time.time()

    #Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    model_inputs = [[query, passage] for passage in passages]
    scores = model.predict(model_inputs)

    #Sort the scores in decreasing order
    results = [{'input': inp, 'score': score} for inp, score in zip(model_inputs, scores)]
    results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("Query:", query)
    print("Search took {:.2f} seconds".format(time.time() - start_time))

    for hit in results:
        if(float(("{:.2f}".format(hit['score']))) > 0):
          matching_tuple = (hit['input'][1], float(("{:.2f}".format(hit['score']))))
          matching_tuples.append(matching_tuple)

    matching_queries_dict[query] = matching_tuples
    matching_posts_count += len(matching_tuples)
    print("Number of Matching Posts:", len(matching_tuples))
    print("==========")

print('\n')
print('For ',len(queries),' Queries, Total Matching Posts: ',matching_posts_count)

In [ ]:
matched_posts_dfs = []
for query_matched in matching_queries_dict.keys():
  matched_tuples = matching_queries_dict[query_matched]
  matched_tuple_df = pd.DataFrame({'Match_Count':([x for x in range(len(matched_tuples))]), 'Processed_Sent':[i[0] for i in matched_tuples], 'Score':[i[1] for i in matched_tuples]})
  matched_tuple_df.sort_values(by=['Score'], ascending=False, inplace=True)
  matched_posts_dfs.append(matched_tuple_df)

In [ ]:
# Find knee in every df and collect all matching posts (Processed_Sent) in a list

matching_posts_list = []

matched_df_count = 0
for matched_post_df in matched_posts_dfs:
  x = matched_post_df['Match_Count']
  y = matched_post_df['Score']
  elbow = -1
  curve_type_direction_list = [('convex','decreasing'), ('convex''increasing'), ('concave','decreasing'), ('concave','increasing')]
  for combination in curve_type_direction_list:
    kn = KneeLocator(x, y, curve=combination[0], direction=combination[1])
    if((kn.knee > 0) & (kn.knee < (len(x)-1))):
      elbow = kn.knee
      print('For DF: ', matched_df_count, ', Eblow at: ', elbow, ' for ', combination[0], '-', combination[1])
      break
  if(elbow > -1):
    current_matching_posts = list(matched_post_df['Processed_Sent'][:elbow])
    matching_posts_list.extend(current_matching_posts)
  matched_df_count += 1

print('\n')
print('Total Number of Matching Posts after Elbow: ', len(matching_posts_list))

In [ ]:
matching_posts_list = list(set(matching_posts_list))
print('Total Number of Unique Matching Posts: ', len(matching_posts_list))

In [ ]:
list(matching_posts_list)

In [ ]:
unique_posts_tuples = []

for x in matching_posts_list:
  post_count = 0
  for y in df['Processed_Sent']:
    if(x == y):
      unique_posts_tuple = ((list(df['post'])[post_count]), x)
      unique_posts_tuples.append(unique_posts_tuple)
    post_count += 1

unique_posts_df = pd.DataFrame({'Post':[i[0] for i in unique_posts_tuples], 'Processed_Sent':[i[1] for i in unique_posts_tuples]})
unique_posts_df.head()

In [ ]:
unique_posts_df.to_csv('Unique_Matching_Posts_Antidodies_Taken.csv', index=False, columns=unique_posts_df.columns)

In [ ]:
posts_containing_ref_to_people_taking_antibodies = list(unique_posts_df['Post'])
processed_sent_containing_ref_to_people_taking_antibodies = list(unique_posts_df['Processed_Sent'])

all_posts = list(df['post'])
all_processed_sent = list(df['Processed_Sent'])

remaining_posts = []
remaining_processed_sent = []

all_posts_count = 0
for x in all_processed_sent:
  if(x not in processed_sent_containing_ref_to_people_taking_antibodies):
    remaining_processed_sent.append(x)
    remaining_posts.append(all_posts[all_posts_count])
  all_posts_count += 1

print('All Posts Count: ', len(all_posts), '\t', len(all_processed_sent))
print('Posts containing antibodies received references Count: ', len(posts_containing_ref_to_people_taking_antibodies), '\t', len(processed_sent_containing_ref_to_people_taking_antibodies))
print('Remaining Posts Count: ', len(remaining_posts), '\t', len(remaining_processed_sent))

In [ ]:
recognizer = absa.aux_models.BasicPatternRecognizer() # TODO - research for better/accurate pattern recognizer
nlp = absa.load('absa/classifier-rest-0.2', pattern_recognizer=recognizer)

In [ ]:
# Get Aspect based sentiments

def get_ab_sentiments_df(posts_list, original_posts_list, aspects_list):
  # PART 1
  post_sentiments_list = []
  posts_count = 0
  for post in posts_list:
    original_post = original_posts_list[posts_count]
    completed_task = nlp(post, aspects=aspects_list)
    total_neutral = 0
    total_positive = 0
    total_negative = 0
    post_sentiments_dict = {}
    for each_aspect in completed_task:
      if(each_aspect.sentiment == absa.Sentiment.positive):
        total_positive += 1
        post_sentiments_dict[each_aspect.aspect] = 'Positive'
      elif(each_aspect.sentiment == absa.Sentiment.negative):
        total_negative += 1
        post_sentiments_dict[each_aspect.aspect] = 'Negative'
      else:
        total_neutral += 1
        post_sentiments_dict[each_aspect.aspect] = 'Neutral'

    overall_sentiment = ''
    if(total_neutral == len(aspects_list)):
      overall_sentiment = 'Neutral'
    elif(total_positive > total_negative):
      overall_sentiment = 'Positive'
    elif(total_negative > total_positive):
      overall_sentiment = 'Negative'
    else:
      overall_sentiment = 'Negative'
    
    post_sentiments_dict['Overall_Sentiment'] = overall_sentiment
    post_sentiments_list.append((original_post, post, post_sentiments_dict))
    posts_count += 1

  # PART 2
  post_sentiments_tuples = []
  # tuple struture: post, 'infusion', 'monoclonal antibodies', 'antibodies', 'regeneron', 'eli lilly', 'bamlanivimab' : use aspects_list

  for post_sentiments_element in post_sentiments_list:
    original_post = post_sentiments_element[0]
    each_post = post_sentiments_element[1]
    post_sentiments_dict = post_sentiments_element[2]

    each_post_tuple = (original_post, each_post,)
    for x in aspects_list:
      for y in post_sentiments_dict.keys():
        if(x == y):
          each_post_tuple = each_post_tuple + (post_sentiments_dict[y],)
          aspect_found = True
          break

    each_post_tuple = each_post_tuple + (post_sentiments_dict['Overall_Sentiment'],) 
    post_sentiments_tuples.append(each_post_tuple)

  # PART 3
  post_sentiments_df = pd.DataFrame({
                                    'Post':[i[0] for i in post_sentiments_tuples],
                                    'Processed_Sent':[i[1] for i in post_sentiments_tuples], 
                                    'Infusion':[i[2] for i in post_sentiments_tuples],
                                    'Monoclonal_Antibodies':[i[3] for i in post_sentiments_tuples],
                                    'Antibodies':[i[4] for i in post_sentiments_tuples],
                                    'Regeneron':[i[5] for i in post_sentiments_tuples],
                                    'Elililly':[i[6] for i in post_sentiments_tuples],
                                    'Bamlanivimab':[i[7] for i in post_sentiments_tuples],
                                    'Overall_Sentiment':[i[8] for i in post_sentiments_tuples]
                                    })

  return post_sentiments_df

In [ ]:
# Get Overall sentiments

def get_overall_sentiments_df(posts_list, original_posts_list):
  posts_count = 0
  post_sentiments_tuples = []
  for post in posts_list:
    original_post = original_posts_list[posts_count]
    polarity = TextBlob(post).sentiment[0]
    subjectivity = TextBlob(post).sentiment[1]

    sentiment = ''
    if(polarity ==0):
      sentiment = 'Neutral'
    elif(polarity < 0):
      sentiment = 'Negative'
    else:
      sentiment = 'Positive'

    each_post_tuple = (original_post, post, polarity, subjectivity, sentiment)
    post_sentiments_tuples.append(each_post_tuple)

  post_sentiments_df = pd.DataFrame({
                                    'Post':[i[0] for i in post_sentiments_tuples],
                                    'Processed_Sent':[i[1] for i in post_sentiments_tuples], 
                                    'Polarity':[i[2] for i in post_sentiments_tuples],
                                    'Subjectivity':[i[3] for i in post_sentiments_tuples],
                                    'Overall_Sentiment':[i[4] for i in post_sentiments_tuples]
                                    })
  
  return post_sentiments_df

In [ ]:
aspects_list = ['infusion', 'monoclonal antibodies', 'antibodies', 'regeneron', 'eli lilly', 'bamlanivimab']

In [ ]:
# Sentiment Analysis (aspect based) related to infusion/antibodies within posts that contain reference to people taking infusion/antibodies

In [ ]:
absa_matching_posts_df = get_ab_sentiments_df(processed_sent_containing_ref_to_people_taking_antibodies, 
                                           posts_containing_ref_to_people_taking_antibodies, 
                                           aspects_list)

absa_matching_posts_df.to_csv('AB_Sentiments_Matching_Posts.csv', index=False, columns=absa_matching_posts_df.columns)

counter_absa_matching_posts=collections.Counter(absa_matching_posts_df['Overall_Sentiment'])
plt.pie(counter_absa_matching_posts.values(), labels=counter_absa_matching_posts.keys(), autopct='%1.1i%%')
plt.title("Sentiments Related to Antibodies/Infusion Within Posts Containing References of People Receiving Antibodies", fontsize=14);
pie.savefig("absa_matching_posts.png")
plt.show()

In [ ]:
# Sentiment Analysis (aspect based) related to infusion/antibodies within REMAINING posts

In [ ]:
absa_remaining_posts_df = get_ab_sentiments_df(remaining_processed_sent, 
                                           remaining_posts, 
                                           aspects_list)

absa_remaining_posts_df.to_csv('AB_Sentiments_Remaining_Posts.csv', index=False, columns=absa_remaining_posts_df.columns)

counter_absa_remaining_posts=collections.Counter(absa_remaining_posts_df['Overall_Sentiment'])
plt.pie(counter_absa_remaining_posts.values(), labels=counter_absa_remaining_posts.keys(), autopct='%1.1i%%')
plt.title("Sentiments Related to Antibodies/Infusion Within Posts Other Than Those Containing References of People Receiving Antibodies", fontsize=14);
pie.savefig("absa_remaining_posts.png")
plt.show()

In [ ]:
# Sentiment Analysis (aspect based) related to infusion/antibodies within ALL posts in corpus

In [ ]:
absa_all_posts_df = get_ab_sentiments_df(all_processed_sent, 
                                           all_posts, 
                                           aspects_list)

absa_all_posts_df.to_csv('AB_Sentiments_All_Posts.csv', index=False, columns=absa_all_posts_df.columns)

counter_absa_all_posts=collections.Counter(absa_all_posts_df['Overall_Sentiment'])
plt.pie(counter_absa_all_posts.values(), labels=counter_absa_all_posts.keys(), autopct='%1.1i%%')
plt.title("Sentiments Related to Antibodies/Infusion Within All Posts", fontsize=14);
pie.savefig("absa_all_posts.png")
plt.show()

In [ ]:
# Sentiment Analysis (overall) of posts containing reference to infusion/antibodies taken

In [ ]:
overall_matching_posts_df = get_overall_sentiments_df(processed_sent_containing_ref_to_people_taking_antibodies, 
                                           posts_containing_ref_to_people_taking_antibodies)

overall_matching_posts_df.to_csv('Overall_Sentiments_Matching_Posts.csv', index=False, columns=overall_matching_posts_df.columns)

counter_overall_matching_posts=collections.Counter(overall_matching_posts_df['Overall_Sentiment'])
plt.pie(counter_overall_matching_posts.values(), labels=counter_overall_matching_posts.keys(), autopct='%1.1i%%')
plt.title("Overall Sentiments Within Posts Containing References of People Receiving Antibodies", fontsize=14);
pie.savefig("overall_matching_posts.png")
plt.show()

In [ ]:
# Sentiment Analysis (overall) of REMANING posts (all - those containing reference to infusion/antibodies taken)

In [ ]:
overall_remaining_posts_df = get_overall_sentiments_df(remaining_processed_sent, 
                                           remaining_posts)

overall_remaining_posts_df.to_csv('Overall_Sentiments_Remaining_Posts.csv', index=False, columns=overall_remaining_posts_df.columns)

counter_overall_remaining_posts=collections.Counter(overall_remaining_posts_df['Overall_Sentiment'])
plt.pie(counter_overall_remaining_posts.values(), labels=counter_overall_remaining_posts.keys(), autopct='%1.1i%%')
plt.title("Overall Sentiments Within Posts Other Than Those Containing References of People Receiving Antibodies", fontsize=14);
pie.savefig("overall_remaining_posts.png")
plt.show()

In [ ]:
# Sentiment Analysis (overall) of ALL posts in corpus

In [ ]:
overall_all_posts_df = get_overall_sentiments_df(all_processed_sent, 
                                           all_posts)

overall_all_posts_df.to_csv('Overall_Sentiments_All_Posts.csv', index=False, columns=overall_all_posts_df.columns)

counter_overall_all_posts=collections.Counter(overall_all_posts_df['Overall_Sentiment'])
plt.pie(counter_overall_all_posts.values(), labels=counter_overall_all_posts.keys(), autopct='%1.1i%%')
plt.title("Overall Sentiments Within All Posts", fontsize=14);
pie.savefig("overall_all_posts.png")
plt.show()